# Kakaomap API

In [1]:
import pandas as pd
import numpy as np
import time

# A simple, yet elegant HTTP library.
import requests
from urllib.parse import urlparse, quote

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm

In [2]:
with open('./config/kakaomaprestapikey.txt', 'r') as f:
    API_KEY = f.read().strip()

In [3]:
df = pd.read_csv('./data/opinet.csv')
df.head(1)

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN


In [4]:
def getCoords(query = ''):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = { 
        'query': query
    }
    headers = { "Authorization": "KakaoAK " + API_KEY }
    api = requests.get(url, params=params, headers=headers).json()
    coords = {}
    coords.update({ 'x': float(api['documents'][0]['x']) })
    coords.update({ 'y': float(api['documents'][0]['y']) })
    return coords

In [5]:
df['lon'] = np.nan # 경도
df['lat'] = np.nan # 위도
for i in tqdm(df.index):
    try:
        addr = df['주소'][i]
        coords = getCoords(addr)
        df['lon'][i] = coords['x']
        df['lat'][i] = coords['y']
    except Exception as e:
        print(i, e)

100%|██████████| 10/10 [00:00<00:00, 21.13it/s]


In [6]:
df.dropna(subset=['lon', 'lat'], inplace=True)
df.index = np.arange(0, len(df))
df.to_csv('./data/kakaomapapi.csv', sep=',', encoding='utf8', index=False)
df

,이름,전화번호,주소,상표,고급휘발유,보통휘발유,경유,실내등유,자동차부탄,lon,lat
0,NaN,02-2695-4544,서울 강서구 화곡로 273 (화곡동),현대오일뱅크,NaN,"1,289","1,147",NaN,NaN,126.846965,37.549279
1,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN,126.914397,37.595614
2,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN,126.913936,37.594096
3,(주)창원이엔지 홍제지점,02-379-0753,서울 서대문구 통일로 455 (홍제동),현대오일뱅크,NaN,"1,299","1,099",NaN,NaN,126.943338,37.589129
4,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN,126.914397,37.595614
5,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN,126.913936,37.594096
6,(주)한진 김포공항주유소,02-2662-0422,서울 강서구 하늘길 259,S-OIL,NaN,"1,319","1,128",NaN,NaN,126.814720,37.550942
7,세원제1주유소,02-3663-5151,서울 강서구 양천로 311 (가양동),SK에너지,NaN,"1,389","1,189",NaN,NaN,126.838076,37.570496
8,다회주유소,02-375-5156,서울 은평구 증산로 441 (신사동),S-OIL,NaN,"1,304","1,104",NaN,NaN,126.914397,37.595614
9,NaN,NaN,서울 은평구 증산로 423 (신사동),GS칼텍스,NaN,"1,315","1,115",NaN,NaN,126.913936,37.594096
